In [1]:
cd /content/drive/MyDrive/DSCNITP/Human Activity Detection

/content/drive/MyDrive/DSCNITP/Human Activity Detection


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Getting Training Data

In [3]:
list_sensors_data=['body_acc_x_train','body_acc_y_train','body_acc_z_train','body_gyro_x_train','body_gyro_y_train',
                   'body_gyro_z_train','total_acc_x_train','total_acc_y_train','total_acc_z_train']

In [14]:
train_data=[]

In [15]:
for fn in list_sensors_data:
  train_data.append(np.array(pd.read_csv("UCI HAR Dataset/train/Inertial Signals/"+fn+".txt",delim_whitespace=True,header=None)))

In [18]:
np.array(train_data).shape

(9, 7352, 128)

In [19]:
np.transpose(train_data,(1,2,0)).shape

(7352, 128, 9)

In [20]:
train_data=np.transpose(train_data,(1,2,0))

In [58]:
train_labels=pd.read_csv("UCI HAR Dataset/train/y_train.txt",header=None)

In [59]:
train_labels=train_labels-1

In [60]:
train_labels=np.array(train_labels)

## Getting Testing Data

In [26]:
list_sensors_data=['body_acc_x_test','body_acc_y_test','body_acc_z_test','body_gyro_x_test','body_gyro_y_test',
                   'body_gyro_z_test','total_acc_x_test','total_acc_y_test','total_acc_z_test']

In [27]:
test_data=[]

In [28]:
for fn in list_sensors_data:
  test_data.append(np.array(pd.read_csv("UCI HAR Dataset/test/Inertial Signals/"+fn+".txt",delim_whitespace=True,header=None)))

In [29]:
np.array(test_data).shape

(9, 2947, 128)

In [30]:
np.transpose(test_data,(1,2,0)).shape

(2947, 128, 9)

In [31]:
test_data=np.transpose(test_data,(1,2,0))

In [61]:
test_labels=pd.read_csv("UCI HAR Dataset/test/y_test.txt",header=None)

In [62]:
test_labels=test_labels-1

In [63]:
test_labels=np.array(test_labels)

## Data Preprocessing

In [64]:
from keras.utils.np_utils import to_categorical

In [65]:
train_labels_cat=to_categorical(train_labels)

In [66]:
test_labels_cat=to_categorical(test_labels)

In [67]:
train_labels_cat.shape

(7352, 6)

# Model Part

In [57]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [68]:
model=Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(128,9)))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

In [69]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 128)          70656     
_________________________________________________________________
dropout (Dropout)            (None, 128, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

# Training the model

In [71]:
model.fit(train_data,train_labels_cat,epochs=10,validation_split=0.05)

Epoch 1/10
219/219 [==============================] - 12s 15ms/step - loss: 1.1174 - accuracy: 0.5243 - val_loss: 0.5801 - val_accuracy: 0.7989
Epoch 2/10
219/219 [==============================] - 2s 11ms/step - loss: 0.5292 - accuracy: 0.8036 - val_loss: 0.4273 - val_accuracy: 0.8288
Epoch 3/10
219/219 [==============================] - 2s 11ms/step - loss: 0.3256 - accuracy: 0.8941 - val_loss: 0.1905 - val_accuracy: 0.8967
Epoch 4/10
219/219 [==============================] - 2s 11ms/step - loss: 0.2063 - accuracy: 0.9273 - val_loss: 0.2248 - val_accuracy: 0.9212
Epoch 5/10
219/219 [==============================] - 2s 11ms/step - loss: 0.1676 - accuracy: 0.9338 - val_loss: 0.2223 - val_accuracy: 0.8913
Epoch 6/10
219/219 [==============================] - 2s 11ms/step - loss: 0.1774 - accuracy: 0.9320 - val_loss: 0.2183 - val_accuracy: 0.8913
Epoch 7/10
219/219 [==============================] - 2s 11ms/step - loss: 0.1411 - accuracy: 0.9440 - val_loss: 0.1293 - val_accuracy: 0.921

# Prediction on testing data

In [78]:
pred_label_cat=model.predict(test_data)

In [79]:
pred_label=np.argmax(pred_label_cat,axis=1)

In [80]:
pred_label

array([4, 4, 4, ..., 1, 1, 1])

# Metrics

In [81]:
import sklearn.metrics as m

In [82]:
m.accuracy_score(test_labels,pred_label)

0.9178825924669155

In [83]:
print(m.classification_report(test_labels,pred_label))

              precision    recall  f1-score   support

           0       0.99      0.94      0.96       496
           1       0.99      0.98      0.99       471
           2       0.94      1.00      0.97       420
           3       0.76      0.86      0.80       491
           4       0.86      0.75      0.80       532
           5       0.99      1.00      0.99       537

    accuracy                           0.92      2947
   macro avg       0.92      0.92      0.92      2947
weighted avg       0.92      0.92      0.92      2947



In [84]:
print(m.confusion_matrix(test_labels,pred_label))

[[467   1  27   1   0   0]
 [  7 461   1   2   0   0]
 [  0   1 419   0   0   0]
 [  0   1   0 420  64   6]
 [  0   0   0 131 401   0]
 [  0   0   0   0   0 537]]


# Saving the model

In [86]:
model.save('model_LSTM.h5')